<a href="https://colab.research.google.com/github/IoannisDem/SimpleDL_models/blob/main/mnist_torch_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms 

In [2]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.deivce('cpu')

In [3]:
#Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [11]:
#create RNN
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    #input_size -> number of features for each time step
    #no need to define sequence length (works it out)
    #hidden size -> number of nodes in each time step
    #dataset will have batches in first axes (N x time_seq x features)
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    #initialise hidden state
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    #Forward prob
    out, _ = self.rnn(x, h0) #dont store hidden state
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)

    return out

In [12]:
#load data

#train_dataset -> (image pixels, label) It has 60000 tuples
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True) #tranforms data to tensors (numpy when loaded)

#splits to batches (938 batches in this case, 60000/64)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

#train_dataset -> (image pixels, label) It has 10000 tuples
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True) #tranforms data to tensors (numpy when loaded)

#splits to batches (157 batches in this case, 10000/64)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [13]:
#stat model
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [14]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()


In [19]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples

In [20]:
check_accuracy(train_loader, model)

tensor(0.9670, device='cuda:0')

In [21]:
check_accuracy(test_loader, model)

tensor(0.9675, device='cuda:0')

In [26]:
#save model
path = F"/content/gdrive/My Drive/Colab Notebooks/models/epic_num_reader_rnn_torch/epic_num_reader_rnn_torch"
torch.save(model.state_dict(), path)

In [ ]:
#load model
model.load_state_dict(torch.load(path))
model.eval()